# Tutorial 17: Custom Visualizations

Advanced matplotlib techniques for physics visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D
from mechanics_dsl import PhysicsCompiler

compiler = PhysicsCompiler()

# Double pendulum for interesting trajectory
dsl_code = r"""
\system{double_pendulum}
\defvar{theta1}{Angle 1}{rad}
\defvar{theta2}{Angle 2}{rad}
\parameter{m1}{1.0}{kg}
\parameter{m2}{1.0}{kg}
\parameter{L1}{1.0}{m}
\parameter{L2}{1.0}{m}
\parameter{g}{9.81}{m/s^2}
\lagrangian{
    \frac{1}{2}*(m1+m2)*L1^2*\dot{theta1}^2 +
    \frac{1}{2}*m2*L2^2*\dot{theta2}^2 +
    m2*L1*L2*\dot{theta1}*\dot{theta2}*\cos{theta1-theta2} +
    (m1+m2)*g*L1*\cos{theta1} + m2*g*L2*\cos{theta2}
}
\initial{theta1=2.0, theta2=2.0, theta1_dot=0.0, theta2_dot=0.0}
"""

compiler.compile_dsl(dsl_code)
sol = compiler.simulate(t_span=(0, 15), num_points=1500)

In [ ]:
t = sol['t']
theta1, theta2 = sol['y'][0], sol['y'][2]
x2 = np.sin(theta1) + np.sin(theta2)
y2 = -np.cos(theta1) - np.cos(theta2)

# Time-colored trajectory
fig, ax = plt.subplots(figsize=(10, 10))
points = np.array([x2, y2]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
lc = LineCollection(segments, cmap='plasma', linewidth=2)
lc.set_array(t)
ax.add_collection(lc)
plt.colorbar(lc, ax=ax, label='Time (s)')
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-2.5, 2.5)
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Time-Colored Trajectory')
ax.set_aspect('equal')
plt.show()

In [ ]:
# Heatmap of trajectory density
fig, ax = plt.subplots(figsize=(8, 8))
h = ax.hist2d(x2, y2, bins=50, cmap='hot')
plt.colorbar(h[3], ax=ax, label='Density')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('Trajectory Density Heatmap')
plt.show()